# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nantucket,41.2835,-70.0995,284.23,79,75,4.12,US,1699280295
1,1,east london,-33.0153,27.9116,295.13,87,100,4.86,ZA,1699280180
2,2,edinburgh of the seven seas,-37.0676,-12.3116,287.08,96,100,11.79,SH,1699280443
3,3,tiksi,71.6872,128.8694,249.47,99,42,3.10,RU,1699280444
4,4,vorkuta,67.5000,64.0000,267.70,95,100,7.34,RU,1699280343


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
city_data_df.hvplot.scatter(x='Lat', y='Lng', c='Humidity', s='Humidity')


:Scatter   [Lat]   (Lng,Humidity)

In [5]:
%%capture

!pip3 install geopandas cartopy geoviews

In [6]:
# city_data_df.hvplot.scatter(x='Lat', y='Lng', c='Humidity', s='Humidity')
scatter_plot = city_data_df.hvplot.scatter(x='Lat', y='Lng', c='Humidity', s='Humidity', cmap='viridis', hover_cols=['City'])

# Render the plot
scatter_plot.opts(xlabel='Latitude', ylabel='Longitude', colorbar=True, width=800, height=600)


:Scatter   [Lat]   (Lng,Humidity,City)

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 25) & (city_data_df["Cloudiness"] < 15) & (city_data_df["Wind Speed"] < 15)]
humidity_max = 55
humidity_min = 15

# Drop any rows with null values
narrow_df.dropna()

city_data_df.dropna(inplace=True)
# Display sample data
narrow_df.head()

print(city_data_df.shape)

display(city_data_df.head())

(109, 10)


/var/folders/l3/l0f9rfpj3258n737tbrhdxkh0000gn/T/ipykernel_47316/3695529472.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_data_df.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,punta arenas,-53.1500,-70.9167,285.21,54,0,3.60,CL,1699280452
16,16,yung shue wan,22.2333,114.1167,300.00,42,32,4.66,HK,1699280459
20,20,yokadouma,3.5167,15.0500,305.40,44,55,1.77,CM,1699280343
23,23,kuacjok,8.3002,27.9903,305.05,44,14,1.37,SS,1699280466
32,32,korla,41.7597,86.1469,280.09,20,8,1.05,CN,1699280476


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
display(hotel_df.head(6))

,City,Country,Lat,Lng,Humidity,Hotel Name
10,punta arenas,CL,-53.1500,-70.9167,54,None
16,yung shue wan,HK,22.2333,114.1167,42,None
20,yokadouma,CM,3.5167,15.0500,44,None
23,kuacjok,SS,8.3002,27.9903,44,None
32,korla,CN,41.7597,86.1469,20,None
46,villa mercedes,AR,-33.6757,-65.4578,22,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    city = row.to_dict()
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
#     params["filter"] = # YOUR CODE HERE
#     params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    params = {
        "lat": city["Lat"],
        "lon": city["Lng"],
        "categories": "accommodation.hotel",
        "radius": radius,  # Search within 10,000 meters (10 km) radius
        "apiKey": geoapify_key
    }

    response = requests.get(base_url, params=params)
    name_address = response.json()


    # Make and API request using the params dictionaty
#     name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
#     name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
punta arenas - nearest hotel: Hotel Lacolet
yung shue wan - nearest hotel: 浪濤軒酒店 Concerto Inn
yokadouma - nearest hotel: Hotel elephant
kuacjok - nearest hotel: Rose Hotel
korla - nearest hotel: Silver Star Hotel
villa mercedes - nearest hotel: La Valle
nouadhibou - nearest hotel: Hotel valencia
puerto natales - nearest hotel: Dorotea Patagonia Hostel
afua - nearest hotel: No hotel found
jwaneng - nearest hotel: Mokala Lodge & Teemane Casino
sucre - nearest hotel: El Gobernador Hostal
constantia - nearest hotel: Hotel Chérica
puerto deseado - nearest hotel: Los Olmos
crane - nearest hotel: Mainstay Suites
aripuana - nearest hotel: Pousada do Rio Roosevelt
jesup - nearest hotel: Days Inn
qaqortoq - nearest hotel: Hotel Qaqortoq
kafr az zayyat - nearest hotel: دلتا ستى مول
porangatu - nearest hotel: No hotel found
gondal - nearest hotel: Hotel Geeta Vihar
silistra - nearest hotel: Силистра
sovetskaya gavan' - nearest hotel: Гостинный Двор
oriximina - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
10,punta arenas,CL,-53.1500,-70.9167,54,Hotel Lacolet
16,yung shue wan,HK,22.2333,114.1167,42,浪濤軒酒店 Concerto Inn
20,yokadouma,CM,3.5167,15.0500,44,Hotel elephant
23,kuacjok,SS,8.3002,27.9903,44,Rose Hotel
32,korla,CN,41.7597,86.1469,20,Silver Star Hotel
...,...,...,...,...,...,...
518,villa gesell,AR,-37.2639,-56.9730,51,Hotel El Faro
529,fisterra,ES,42.9049,-9.2629,51,Hotel Temático do Banco Azul
540,namibe,AO,-15.1961,12.1522,52,Hotel Chik Chik Namibe
543,dalandzadgad,MN,43.5708,104.4250,24,Хан Уул буудал


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [17]:
# https://hvplot.holoviz.org/user_guide/Geographic_Data.html
city_data_df.hvplot.points('Lng', 'Lat', geo=True, color='blue', alpha=0.2, s='Humidity', tiles='Hotel Name')


UnboundLocalError: cannot access local variable 'tiles' where it is not associated with a value